In [2]:
pip install pennylane==0.26

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 1.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 69.7 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 9.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 80.1 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 11.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 12.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 10.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 10.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 5.3 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 5.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 4.9 MB/s eta 0:00:00:00:0100:01
     ━━━

In [36]:
pip install pennylane-qiskit==0.24.0


[notice] A new release of pip available: 23.1.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pennylane as qml

dev = qml.device('default.qubit',wires = 2)

def qc():
    #와이어는 정수로 입력하고 멀티큐빗게이트는 리스트 형태로 입력
    qml.PauliX(wires=0)
    qml.Hadamard(wires=0)
    return qml.state()

#회로와 장치를 합함
qcirc = qml.QNode(qc,dev)
qcirc()

tensor([ 0.70710678+0.j,  0.        +0.j, -0.70710678+0.j,
         0.        +0.j], requires_grad=True)

In [2]:
#어노테이션을 이용해 합치기 가능
#qml.state() = state vector
#qml.probs(wires=w) w라인의 확률진폭

@qml.qnode(dev)
def qcirc():
    qml.PauliX(wires = 0)
    qml.Hadamard(wires=0)
    return qml.state()
qcirc()

tensor([ 0.70710678+0.j,  0.        +0.j, -0.70710678+0.j,
         0.        +0.j], requires_grad=True)

In [3]:
dev = qml.device('default.qubit',wires=1)
@qml.qnode(dev)
def qcirc(theta):
    qml.RX(theta,wires=0)
    return qml.state()


In [4]:
print(qml.draw(qcirc)(theta=2))

0: ──RX(2.00)─┤  State


In [5]:
dev = qml.device('default.qubit',wires=3)

#확률 얻기
@qml.qnode(dev)
def qcirc():
    qml.Hadamard(wires = 1)
    return qml.probs(wires = [1,2])
prob = qcirc()
print("Probs. wires[1,2] with H in wire 1 : ",prob)

#shot을 정하지 않은 샘플
#실행시 shot을 설정해줘야 함
qml.qnode(dev)
def circ():
    qml.Hadamard(wires=0)
    return qml.smaple(wires=0)
s1 = qcirc(shots=4)
print("sample 1 after H :",s1)

#shot을 정한 샘플
dev = qml.device('default.qubit',wires=2,shots=4)
@qml.qnode(dev)
def qcirc():
    qml.Hadamard(wires=0)
    return qml.sample()
s2 = qcirc()
print("sample 2 after H X I :",s2)

Probs. wires[1,2] with H in wire 1 :  [0.5 0.  0.5 0. ]
sample 1 after H : [0.5 0.  0.5 0. ]
sample 2 after H X I : [[1 0]
 [0 0]
 [0 0]
 [1 0]]


In [6]:
#example 2-5
import numpy as np

dev = qml.device('default.qubit',wires=2)
@qml.qnode(dev)
def qcirc():
    qml.PauliZ(wires=0)
    qml.PauliY(wires=1)
    qml.CRY(np.pi/2,wires=[0,1])
    qml.U3(np.pi/4,np.pi,0,0)
    qml.RZ(phi=np.pi/4,wires=1)
    return qml.state()
print(qcirc())

[ 0.        +0.j         -0.35355339+0.85355339j  0.        +0.j
  0.14644661-0.35355339j]


In [13]:
#qiskit.aer를 사용해 시뮬레이션 하기

dev = qml.device('qiskit.aer',wires=2,backend='aer_simulator_statevector',shots=None)
@qml.qnode(dev)
def qcirc():
    qml.Hadamard(wires=0)
    return qml.state()
s = qcirc()
print("statevector : ",s)

statevector :  [0.70710678+0.j 0.        +0.j 0.70710678+0.j 0.        +0.j]


In [ ]:
#IBMQ와 연결하기

from qiskit import *
from qiskit.providers.ibmq import *

bck = least_busy(backends(simulator=False)).name()

dev = qml.device('qiskit.ibmq',wires=1,backend=bck,provider=prov)

#회로 IBMQ에 전송하여 결과 받기
@qml.qnode(dev)
def qcirc():
    qml.Hadamard(wires=0)
    return qml.probs(wires=0)
print(qcirc())